In [1]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Database connection
conn = psycopg2.connect(
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    database=os.getenv('DB_NAME')
)

print("✅ Successfully connected to grocery_sales_db database!")

✅ Successfully connected to grocery_sales_db database!


# Practical Exam: Grocery Store Sales

FoodYum is a grocery store chain that is based in the United States.

Food Yum sells items such as produce, meat, dairy, baked goods, snacks, and other household food staples.

As food costs rise, FoodYum wants to make sure it keeps stocking products in all categories that cover a range of prices to ensure they have stock for a broad range of customers. 

## Data

The data is available in the table `products`.

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with 2022. |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

# Task 1

Last year (2022) there was a bug in the product system. For some products that were added in that year, the `year_added` value was not set in the data. As the year the product was added may have an impact on the price of the product, this is important information to have. 

Write a query to determine how many products have the `year_added` value missing. Your output should be a single column, `missing_year`, with a single row giving the number of missing values.

In [2]:
# Task 1: Count missing year_added values
query = """
SELECT COUNT(*) AS missing_year
FROM products
WHERE year_added IS NULL;
"""

df_missing_year = pd.read_sql_query(query, conn)
print(f"\n📊 Products with missing year_added: {df_missing_year['missing_year'][0]}")
df_missing_year


📊 Products with missing year_added: 170


C:\Users\Asus\AppData\Local\Temp\ipykernel_16472\1234536702.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_missing_year = pd.read_sql_query(query, conn)


,missing_year
0,170


# Task 2

Given what you know about the year added data, you need to make sure all of the data is clean before you start your analysis. The table below shows what the data should look like. 

Write a query to ensure the product data matches the description provided. Do not update the original table.  

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with last year (2022). |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

In [3]:
# Task 2: Comprehensive data cleaning
query = """
SELECT
    product_id,

    -- Clean and standardize product_type
    CASE
        WHEN TRIM(LOWER(product_type)) IN ('produce') THEN 'Produce'
        WHEN TRIM(LOWER(product_type)) IN ('meat') THEN 'Meat'
        WHEN TRIM(LOWER(product_type)) IN ('dairy') THEN 'Dairy'
        WHEN TRIM(LOWER(product_type)) IN ('bakery') THEN 'Bakery'
        WHEN TRIM(LOWER(product_type)) IN ('snacks') THEN 'Snacks'
        ELSE 'Unknown'
    END AS product_type,

    -- Clean and handle brand
    CASE
        WHEN TRIM(LOWER(brand)) IN ('', '-', 'missing') OR brand IS NULL THEN 'Unknown'
        ELSE TRIM(brand)
    END AS brand,

    -- Clean and round weight
    ROUND((
        COALESCE(
            NULLIF(REGEXP_REPLACE(weight, '[^0-9\.]', '', 'g'), '')::NUMERIC,
            (
                SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (
                    ORDER BY NULLIF(REGEXP_REPLACE(weight, '[^0-9\.]', '', 'g'), '')::NUMERIC
                )
                FROM products
                WHERE NULLIF(REGEXP_REPLACE(weight, '[^0-9\.]', '', 'g'), '') IS NOT NULL
            )
        )
    )::NUMERIC, 2) AS weight,

    -- Clean and round price
    ROUND((
        COALESCE(
            NULLIF(price::TEXT, '')::NUMERIC,
            (
                SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (
                    ORDER BY price::NUMERIC
                )
                FROM products
                WHERE price IS NOT NULL AND price::TEXT <> ''
            )
        )
    )::NUMERIC, 2) AS price,

    -- Replace NULL or non-numeric average_units_sold with 0
    COALESCE(average_units_sold, 0) AS average_units_sold,

    -- Replace missing year_added with 2022
    COALESCE(year_added, '2022') AS year_added,

    -- Clean and standardize stock_location
    CASE
        WHEN TRIM(LOWER(stock_location)) IN ('a','b','c','d') THEN UPPER(TRIM(stock_location))
        ELSE 'Unknown'
    END AS stock_location

FROM 
    products;
"""

df_cleaned = pd.read_sql_query(query, conn)
print(f"\n📊 Cleaned products table: {len(df_cleaned)} rows")
df_cleaned.head(10)

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Asus\AppData\Local\Temp\ipykernel_16472\3575248830.py:2: SyntaxWarning: invalid escape sequence '\.'
  query = """
C:\Users\Asus\AppData\Local\Temp\ipykernel_16472\3575248830.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cleaned = pd.read_sql_query(query, conn)



📊 Cleaned products table: 1700 rows


,product_id,product_type,brand,weight,price,average_units_sold,year_added,stock_location
0,1,Bakery,TopBrand,602.61,11.00,15,2022,C
1,2,Produce,SilverLake,478.26,8.08,22,2022.0,C
2,3,Produce,TastyTreat,532.38,6.16,21,2018.0,B
3,4,Bakery,StandardYums,453.43,7.26,21,2021.0,D
4,5,Produce,GoldTree,588.63,7.88,21,2020.0,A
5,6,Meat,TopBrand,612.06,16.20,24,2017.0,A
6,7,Produce,GoldTree,320.49,8.01,21,2019.0,B
7,8,Meat,SilverLake,535.19,15.77,28,2021.0,A
8,9,Meat,StandardYums,375.07,11.57,30,2020.0,A
9,10,Meat,TastyTreat,506.34,13.94,27,2018.0,C


# Task 3

To find out how the range varies for each product type, your manager has asked you to determine the minimum and maximum values for each product type.   

Write a query to return the `product_type`, `min_price` and `max_price` columns. 

In [4]:
# Task 3: Price range by product type
query = """
SELECT 
    COALESCE(product_type, 'Unknown') AS product_type,
    MIN(price) AS min_price,
    MAX(price) AS max_price
FROM 
    products
GROUP BY 
    COALESCE(product_type, 'Unknown')
ORDER BY product_type;
"""

df_price_range = pd.read_sql_query(query, conn)
print(f"\n📊 Price ranges across {len(df_price_range)} product categories:")
df_price_range


📊 Price ranges across 5 product categories:


C:\Users\Asus\AppData\Local\Temp\ipykernel_16472\2239315538.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_price_range = pd.read_sql_query(query, conn)


,product_type,min_price,max_price
0,Bakery,6.26,11.88
1,Dairy,8.33,13.97
2,Meat,11.48,16.98
3,Produce,3.46,8.78
4,Snacks,5.20,10.72


# Task 4

The team want to look in more detail at meat and dairy products where the average units sold was greater than ten. 

Write a query to return the `product_id`, `price` and `average_units_sold` of the rows of interest to the team. 

In [5]:
# Task 4: High-demand meat and dairy products
query = """
SELECT 
    product_id, 
    price, 
    average_units_sold
FROM 
    products
WHERE 
    product_type IN ('Meat', 'Dairy')
    AND average_units_sold > 10
ORDER BY average_units_sold DESC;
"""

df_high_demand = pd.read_sql_query(query, conn)
print(f"\n📊 High-demand meat & dairy products (>10 units/month): {len(df_high_demand)} products")
df_high_demand.head(10)


📊 High-demand meat & dairy products (>10 units/month): 698 products


C:\Users\Asus\AppData\Local\Temp\ipykernel_16472\2556315798.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_high_demand = pd.read_sql_query(query, conn)


,product_id,price,average_units_sold
0,138,11.79,31
1,733,11.76,31
2,307,12.52,31
3,996,12.00,31
4,189,12.92,31
5,1137,12.19,31
6,510,12.38,31
7,480,12.10,31
8,804,12.10,31
9,165,12.89,30
